## Tratamiento de datos para modelo de evolución bursátil por fundamentales

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 50

Los ficheros que vamos a tratar se han obtenido de la web simfin https://simfin.com/data/bulk, que proporciona datos de compañías cotizadas en EEUU desde 2007.
Nos descargamos 5 ficheros (10_6_20) con los datos de cotización, industrias, datos de compañías, balance y cuenta de resultados. Estas dos últimas solo para bancos, ya que el formato de sus cuentas es diferente al resto de compañías.

In [3]:
! ls -1 ../datos

DJUSFN.csv
industries.csv
us-balance-banks-quarterly.csv
us-companies.csv
us-income-banks-quarterly.csv
us-shareprices-daily.csv


In [4]:
! head ../datos/us-companies.csv

Ticker;SimFinId;"Company Name";IndustryId
GOOG;18;Alphabet;101002
FOX_old;242;"21st Century Fox";103001
ATVI;243;"Activision Blizzard";101003
ADBE;14099;"Adobe Systems";101003
MMM;22219;"3M CO";100001
TWTR;30388;"TWITTER, INC.";101002
BRID;33021;"BRIDGFORD FOODS CORP";102002
NWY;33184;"New York & Company, Inc.";103002
AMSC;33387;"AMERICAN SUPERCONDUCTOR CORP /DE/";101001


In [5]:
! head ../datos/us-shareprices-daily.csv

Ticker;SimFinId;Date;Open;Low;High;Close;"Adj. Close";Dividend;Volume
GOOG;18;2007-01-03;115.9490;114.7323;118.6014;116.3447;232.92;;15470700
GOOG;18;2007-01-04;116.6955;116.5337;120.4153;120.2436;240.73;;15834200
GOOG;18;2007-01-05;120.0545;118.9622;121.2986;121.2214;242.69;;13795600
GOOG;18;2007-01-08;121.3459;119.9799;121.8883;120.3232;240.89;;9544400
GOOG;18;2007-01-09;120.7885;119.7310;121.4852;120.8009;241.84;;10803000
GOOG;18;2007-01-10;120.5347;119.9401;122.8039;121.7863;243.82;;11981700
GOOG;18;2007-01-11;123.7121;123.4583;124.8443;124.3392;248.93;;14470400
GOOG;18;2007-01-12;124.9039;124.4088;125.6529;125.6529;251.56;;8980800
GOOG;18;2007-01-16;126.2874;125.2299;127.6435;125.4737;251.20;;15194500


In [6]:
balance = pd.read_csv('../datos/us-balance-banks-quarterly.csv', sep=';')
resultados = pd.read_csv('../datos/us-income-banks-quarterly.csv', sep=';', 
                         usecols=['Ticker','Fiscal Year','Fiscal Period', 'Revenue',
                                  'Provision for Loan Losses', 'Net Revenue after Provisions',
                                  'Total Non-Interest Expense', 'Operating Income (Loss)',
                                  'Non-Operating Income (Loss)', 'Pretax Income (Loss)',
                                  'Income Tax (Expense) Benefit, Net', 'Income (Loss) from Continuing Operations',
                                  'Net Extraordinary Gains (Losses)', 'Net Income', 'Net Income (Common)'])
industries = pd.read_csv('../datos/industries.csv', sep=';', dtype='str')
companies = pd.read_csv('../datos/us-companies.csv', sep=';', dtype='str')
cotizacion = pd.read_csv('../datos/us-shareprices-daily.csv', sep=';', date_parser='Dates')
# Utilizamos dtype en companies e industries para convertir los códigos numéricos en caracteres.

In [7]:
#Identificamos el código industrial 'IndustryId' correspondiente al sector bancario
industries.Sector.unique()

array(['Industrials', 'Technology', 'Consumer Defensive',
       'Consumer Cyclical', 'Financial Services', 'Utilities',
       'Healthcare', 'Energy', 'Business Services', 'Real Estate',
       'Basic Materials', 'Other'], dtype=object)

In [8]:
industries[industries.Sector=='Financial Services']

,IndustryId,Sector,Industry
35,104001,Financial Services,Asset Management
36,104002,Financial Services,Banks
37,104003,Financial Services,Brokers & Exchanges
38,104004,Financial Services,Insurance - Life
39,104005,Financial Services,Insurance
40,104006,Financial Services,Insurance - Property & Casualty
41,104007,Financial Services,Credit Services
42,104013,Financial Services,Insurance - Specialty


In [9]:
companies.head(1)

,Ticker,SimFinId,Company Name,IndustryId
0,GOOG,18,Alphabet,101002


In [10]:
# De la tabla de compañías, nos quedamos únicamente con los bancos
banks = companies[companies.IndustryId=='104002']
print(banks.count())
banks.head()

Ticker          57
SimFinId        57
Company Name    57
IndustryId      57
dtype: int64


,Ticker,SimFinId,Company Name,IndustryId
91,GS,60439,GOLDMAN SACHS GROUP INC,104002
138,BAC,71941,BANK OF AMERICA CORP /DE/,104002
190,C,89126,CITIGROUP INC,104002
330,IBKC,129297,IBERIABANK CORP,104002
331,NKSH,129536,NATIONAL BANKSHARES INC,104002


In [11]:
cotizacion.head()

,Ticker,SimFinId,Date,Open,Low,High,Close,Adj. Close,Dividend,Volume
0,GOOG,18,2007-01-03,115.9490,114.7323,118.6014,116.3447,232.92,NaN,15470700
1,GOOG,18,2007-01-04,116.6955,116.5337,120.4153,120.2436,240.73,NaN,15834200
2,GOOG,18,2007-01-05,120.0545,118.9622,121.2986,121.2214,242.69,NaN,13795600
3,GOOG,18,2007-01-08,121.3459,119.9799,121.8883,120.3232,240.89,NaN,9544400
4,GOOG,18,2007-01-09,120.7885,119.7310,121.4852,120.8009,241.84,NaN,10803000


In [29]:
#Filtramos la tabla de cotizaciones únicamente con los bancos y exploramos un poco el resultado para ver
# cuantas compañías disponen de cotizaciones diarias para la serie completa 2007-01-03 -- 2019-06-07

banks_cotiz = cotizacion[cotizacion.Ticker.isin(banks.Ticker)]

# Tenemos que reiniciar el índice
banks_cotiz.reset_index(inplace=True)
banks_cotiz=banks_cotiz.drop('index', axis=1)
# Exportamos esta tabla para más adelante
banks_cotiz.to_csv('../tablas/banks_cotiz.csv', index=False)

# Vemos que son 43 bancos
banks_cotiz.groupby('Ticker').count()[banks_cotiz.groupby('Ticker').count().Close == 3129].count()

SimFinId      43
Date          43
Open          43
Low           43
High          43
Close         43
Adj. Close    43
Dividend      43
Volume        43
dtype: int64

Con 43 bancos desde enero del 2007 hasta junio de 2019, si hacemos ventanas temporales trimestrales, en principio contaríamos con una población para alimentar el modelo de (12*4 + 1)*43 = 2.107

A continuación verificamos la disponibilidad de información contable para estos bancos.

In [13]:
# Creamos una nueva tabla de datos de balance con los bancos para los cuales disponemos cotización
balance_filtrado = balance[balance['Ticker'].isin(banks_cotiz.Ticker)]

In [14]:
# Aquí contamos el número de datos trimestrales disponibles.
exp = balance_filtrado.groupby('Ticker').Currency.count()
len(exp[exp>31])

35

In [15]:
# Creamos una nueva tabla de datos de pyg con los bancos para los cuales disponemos cotización
resultados_filtrado = resultados[resultados['Ticker'].isin(banks_cotiz.Ticker)]

In [16]:
# Aquí contamos el número de datos trimestrales disponibles, obteniendo 35 empresas con 31 datos trimestrales.
# Utilizando solo estos datos dispondríamos de una población de 1.085 registros.
exp = resultados_filtrado.groupby('Ticker').Revenue.count()
len(exp[exp>31])

35

A continuación, debemos generar una tabla única en la que se encuentre incorporado el mayor número posible de datos contables y con el dato de cotización al final del periodo. Aquí tenemos una limitación, y es que en realidad, la información contable al cierre de un trimestre no está disponible en ese mismo momento. Como simplificación del modelo podemos utilizar la cotización del final del periodo, o, alternativamente, utilizar la fecha de report (!que está disponible!). En cualquiera de las dos alternativas, el plazo de análisis de evolución de la cotización será de tres meses.

### Generar tabla principal de variables
En esta tabla cada registro representa un valor en un periodo determinado. Podemos partir de balance_filtrado y crear una referencia única a partir de Ticker, Fiscal Year y Fiscal Period.

In [17]:
balance_filtrado = balance_filtrado.astype({'Fiscal Year':str})

In [18]:
balance_filtrado['Ref'] = balance_filtrado['Ticker'] + balance_filtrado['Fiscal Year'] + balance_filtrado['Fiscal Period']

In [19]:
balance_filtrado.index = balance_filtrado.Ref
balance_filtrado.head()

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),"Cash, Cash Equivalents & Short Term Investments",Interbank Assets,Short & Long Term Investments,Accounts & Notes Receivable,Net Loans,Net Fixed Assets,Total Assets,Total Deposits,Short Term Debt,Long Term Debt,Total Liabilities,Preferred Equity,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity,Ref
Ref,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BAC2011Q3,BAC,71941,USD,2011,Q3,2011-09-30,2011-11-03,2011-11-03,1.011628e+10,1.046440e+10,82865000000,2.499980e+11,624497000000,NaN,8.974490e+11,1.355200e+10,2219628000000,1.041353e+12,3.500110e+11,3.989650e+11,1989376000000,1.948000e+10,153801000000,NaN,5.904300e+10,230252000000,2219628000000,BAC2011Q3
BAC2011Q4,BAC,71941,USD,2011,Q4,2011-12-31,2012-02-23,2013-02-28,1.028341e+10,1.027862e+10,120102000000,2.111830e+11,579762000000,NaN,8.924170e+11,1.363700e+10,2129046000000,1.033041e+12,3.110700e+11,3.722650e+11,1898945000000,1.839700e+10,156621000000,NaN,6.052000e+10,230101000000,2129046000000,BAC2011Q4
BAC2012Q1,BAC,71941,USD,2012,Q1,2012-03-31,2012-05-03,2012-05-03,1.065137e+10,1.076192e+10,128792000000,2.257840e+11,620550000000,NaN,8.700830e+11,1.310400e+10,2181449000000,1.041311e+12,3.681590e+11,3.549120e+11,1948950000000,1.878800e+10,157973000000,NaN,6.073400e+10,232499000000,2181449000000,BAC2012Q1
BAC2012Q2,BAC,71941,USD,2012,Q2,2012-06-30,2012-08-02,2012-08-02,1.077570e+10,1.155601e+10,123717000000,2.261160e+11,622231000000,NaN,8.620270e+11,1.265300e+10,2160854000000,1.035225e+12,4.023910e+11,3.018480e+11,1924879000000,1.876200e+10,158001000000,NaN,6.271200e+10,235975000000,2160854000000,BAC2012Q2
BAC2012Q3,BAC,71941,USD,2012,Q3,2012-09-30,2012-11-02,2012-11-02,1.077617e+10,1.077617e+10,106415000000,2.340340e+11,630752000000,NaN,8.668020e+11,1.243600e+10,2166162000000,1.063307e+12,3.813700e+11,2.865340e+11,1927556000000,1.876800e+10,158066000000,NaN,6.258300e+10,238606000000,2166162000000,BAC2012Q3


In [20]:
resultados_filtrado = resultados_filtrado.astype({'Fiscal Year':str})

In [21]:
resultados_filtrado['Ref'] = resultados_filtrado['Ticker'] + resultados_filtrado['Fiscal Year'] + resultados_filtrado['Fiscal Period']

In [22]:
resultados_filtrado.index = resultados_filtrado.Ref
resultados_filtrado.head()

,Ticker,Fiscal Year,Fiscal Period,Revenue,Provision for Loan Losses,Net Revenue after Provisions,Total Non-Interest Expense,Operating Income (Loss),Non-Operating Income (Loss),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common),Ref
Ref,,,,,,,,,,,,,,,,
BAC2011Q3,BAC,2011,Q3,28453000000,-3.407000e+09,25046000000,-17437000000,7609000000,NaN,7433000000,-1.201000e+09,6232000000,NaN,6232000000,5889000000,BAC2011Q3
BAC2011Q4,BAC,2011,Q4,24888000000,-2.934000e+09,21954000000,-18840000000,3114000000,NaN,2432000000,-4.410000e+08,1991000000,NaN,1991000000,1584000000,BAC2011Q4
BAC2012Q1,BAC,2012,Q1,22278000000,-2.418000e+09,19860000000,-19141000000,719000000,NaN,719000000,-6.600000e+07,653000000,NaN,653000000,328000000,BAC2012Q1
BAC2012Q2,BAC,2012,Q2,21968000000,-1.773000e+09,20195000000,-17048000000,3147000000,NaN,3147000000,-6.840000e+08,2463000000,NaN,2463000000,2098000000,BAC2012Q2
BAC2012Q3,BAC,2012,Q3,20428000000,-1.774000e+09,18654000000,-17544000000,1110000000,NaN,1110000000,-7.700000e+08,340000000,NaN,340000000,-33000000,BAC2012Q3


### Merge de tablas balance y resultados

In [23]:
balance_filtrado.drop('Ref', axis=1, inplace=True)
resultados_filtrado.drop(['Ref','Fiscal Year', 'Fiscal Period','Ticker'], axis=1, inplace=True)

In [24]:
mergedbalres=balance_filtrado.merge(resultados_filtrado, on= 'Ref', how='left')

In [25]:
mergedbalres.head(5)

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),"Cash, Cash Equivalents & Short Term Investments",Interbank Assets,Short & Long Term Investments,Accounts & Notes Receivable,Net Loans,Net Fixed Assets,Total Assets,Total Deposits,Short Term Debt,Long Term Debt,Total Liabilities,Preferred Equity,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity,Revenue,Provision for Loan Losses,Net Revenue after Provisions,Total Non-Interest Expense,Operating Income (Loss),Non-Operating Income (Loss),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
Ref,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BAC2011Q3,BAC,71941,USD,2011,Q3,2011-09-30,2011-11-03,2011-11-03,1.011628e+10,1.046440e+10,82865000000,2.499980e+11,624497000000,NaN,8.974490e+11,1.355200e+10,2219628000000,1.041353e+12,3.500110e+11,3.989650e+11,1989376000000,1.948000e+10,153801000000,NaN,5.904300e+10,230252000000,2219628000000,28453000000,-3.407000e+09,25046000000,-17437000000,7609000000,NaN,7433000000,-1.201000e+09,6232000000,NaN,6232000000,5889000000
BAC2011Q4,BAC,71941,USD,2011,Q4,2011-12-31,2012-02-23,2013-02-28,1.028341e+10,1.027862e+10,120102000000,2.111830e+11,579762000000,NaN,8.924170e+11,1.363700e+10,2129046000000,1.033041e+12,3.110700e+11,3.722650e+11,1898945000000,1.839700e+10,156621000000,NaN,6.052000e+10,230101000000,2129046000000,24888000000,-2.934000e+09,21954000000,-18840000000,3114000000,NaN,2432000000,-4.410000e+08,1991000000,NaN,1991000000,1584000000
BAC2012Q1,BAC,71941,USD,2012,Q1,2012-03-31,2012-05-03,2012-05-03,1.065137e+10,1.076192e+10,128792000000,2.257840e+11,620550000000,NaN,8.700830e+11,1.310400e+10,2181449000000,1.041311e+12,3.681590e+11,3.549120e+11,1948950000000,1.878800e+10,157973000000,NaN,6.073400e+10,232499000000,2181449000000,22278000000,-2.418000e+09,19860000000,-19141000000,719000000,NaN,719000000,-6.600000e+07,653000000,NaN,653000000,328000000
BAC2012Q2,BAC,71941,USD,2012,Q2,2012-06-30,2012-08-02,2012-08-02,1.077570e+10,1.155601e+10,123717000000,2.261160e+11,622231000000,NaN,8.620270e+11,1.265300e+10,2160854000000,1.035225e+12,4.023910e+11,3.018480e+11,1924879000000,1.876200e+10,158001000000,NaN,6.271200e+10,235975000000,2160854000000,21968000000,-1.773000e+09,20195000000,-17048000000,3147000000,NaN,3147000000,-6.840000e+08,2463000000,NaN,2463000000,2098000000
BAC2012Q3,BAC,71941,USD,2012,Q3,2012-09-30,2012-11-02,2012-11-02,1.077617e+10,1.077617e+10,106415000000,2.340340e+11,630752000000,NaN,8.668020e+11,1.243600e+10,2166162000000,1.063307e+12,3.813700e+11,2.865340e+11,1927556000000,1.876800e+10,158066000000,NaN,6.258300e+10,238606000000,2166162000000,20428000000,-1.774000e+09,18654000000,-17544000000,1110000000,NaN,1110000000,-7.700000e+08,340000000,NaN,340000000,-33000000


In [26]:
mergedbalres.to_csv('../tablas/mergedbalres.csv')

El siguiente paso en nuestro proyecto consiste en el etiquetado de cada uno de los registros de la tabla mergedbalres en función de su comportamiento en relación a un índice de mercado.
Abordamos esta fase en el notebook 'etiquetado'.